# Generar piezas musicales con el dataset de Spanio 🎵

Utilizar los pesos de music_audioset_epoch_15_esc_90.14.pt en lugar de los genéricos que carga clap_model.load_ckpt() para verificar si hay una mejoría en el CLAP Score.

In [1]:
import os
import pandas as pd
import numpy as np

import torch
import torchaudio
from laion_clap import CLAP_Module
from torch import serialization
from tqdm import tqdm

import laion_clap.clap_module.factory as factory
import laion_clap.hook as hook

/home/juana/.venv311/lib64/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import sys
from pathlib import Path

# Detectar la raíz del proyecto automáticamente
PROJECT_ROOT = Path.cwd()
for parent in [PROJECT_ROOT, *PROJECT_ROOT.parents]:
    if (parent / "models" / "scripts" / "types.py").exists():
        PROJECT_ROOT = parent
        break

# Agregar la raíz al sys.path si no está
if str(PROJECT_ROOT) not in sys.path:
    sys.path.append(str(PROJECT_ROOT))

print("🔍 Raíz del proyecto:", PROJECT_ROOT)


🔍 Raíz del proyecto: /home/juana/audio_reprompt


In [10]:
from models.scripts.types import MusicGenCLAPResult, MusicGenData
from config import load_config, setup_project_paths, PROJECT_ROOT

In [4]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Dispositivo: {DEVICE}")

Dispositivo: cpu


In [11]:
print("Configurando las rutas del proyecto...")
setup_project_paths()

print("Cargando configuración...")
config = load_config()

Configurando las rutas del proyecto...
Cargando configuración...


In [14]:
tracks_base_data_path = PROJECT_ROOT / config.data.tracks_base_data_path
data_clap_path = (
    PROJECT_ROOT / config.data.data_clap_path / "results_with_clap_weights.csv"
)

laion_clap_path = PROJECT_ROOT / config.model.laion_clap_path

### Calcular el CLAP Score

`CLAP_Module`:

Es un wrapper del modelo CLAP que permite el uso del modelo CLAP para obtener embeddings de audio y texto, calcular similitudes o entrenar nuevos modelos multimodales.

Parámetro:

- `enable_fusion`: activa o desactiva un mecanismo interno del modelo CLAP que combina información de audio y texto en una representación conjunta, es decir, un embedding fusionado. Esto permite calcular de forma directa una similitud entre embeddings audio ↔ texto, sin necesidad de entrenar un modelo adicional. Si es False, el modelo cargaría solo el codificador de audio o texto, sin capacidad de comparar entre ellos, por lo que el cálculo de similitud coseno no tendría sentido.

Fórmula de la similitud del coseno:

$$
\text{sim}(a, b) = \frac{a \cdot b}{\|a\| \|b\|}
$$

El resultado (score) es un número entre -1 y 1:

- +1 → audio y texto son muy similares.

- 0 → no hay relación.

- -1 → son opuestos semánticamente (raro en práctica).

¿Por qué en CLAP casi nunca salen negativos?

El modelo CLAP fue entrenado con una pérdida contrastiva tipo InfoNCE que:

- Maximiza la similitud entre los pares correctos (audio ↔ descripción) 
- Minimiza la similitud entre los pares incorrectos.

El modelo nunca vio ejemplos de “oposición semántica” (como “silencio” vs “explosión”) durante el entrenamiento, por eso, el coseno rara vez llega a valores extremos (−1 o 1).

Por lo tanto, tras el entrenamiento:

| Tipo de relación audio-texto | CLAP Score típico |
| ---------------------------- | ----------------- |
| Muy alta coherencia          | 0.7 – 0.9         |
| Moderada coherencia          | 0.4 – 0.6         |
| Poca coherencia              | 0.2 – 0.4         |
| Ruido o sin relación         | < 0.2             |

[Ver referencia de clap score](https://arxiv.org/html/2506.23553v2)

#### Funciones para utilizar otros pesos con Clap Score

El modelo CLAP (Contrastive Language-Audio Pretraining) tiene múltiples variantes entrenadas en distintos datasets:

| Alias            | Modelo base real    | Dataset                 | Uso principal            |
| ---------------- | ------------------- | ----------------------- | ------------------------ |
| `music_audioset` | `HTSAT-base`        | AudioSet + ESC-50       | Música y sonidos         |
| `HTSAT-base`     | Transformer HTSAT   | Audio feature extractor | Base de “music_audioset” |
| `roberta-base`   | Transformer textual | Text encoder            | Texto de prompts         |


El checkpoint music_audioset_epoch_15_esc_90.14.pt fue entrenado sobre la arquitectura HTSAT-base (audio encoder) + roberta-base (text encoder), pero el código original del paquete laion_clap no reconoce “music_audioset” como un nombre de modelo válido.

¿Por qué no poner directamente HTSAT-base?

Porque el nombre music_audioset no es solo un alias cosmético, sino que está ligado al tipo de checkpoint y la arquitectura que CLAP espera internamente.

- HTSAT-base es un modelo de audio genérico.

- music_audioset es una versión fine-tuned (ajustada) de HTSAT-base + text encoder (RoBERTa) sobre el dataset AudioSet + ESC-50.

In [ ]:
def patched_create_model(amodel_name: str, *args, **kwargs):
    """
    Intercepta la llamada que crea el modelo y reemplaza el identificador "music_audioset" por 
    "HTSAT-base" antes de que el código interno lo procese.
    """
    if amodel_name == "music_audioset":
        print("🎵 Usando modelo 'music_audioset' (alias de HTSAT-base)")
        amodel_name = "HTSAT-base"
    return (
        factory._create_model(amodel_name, *args, **kwargs)
        if hasattr(factory, "_create_model")
        else factory.create_model(amodel_name, *args, **kwargs)
    )


# Guardar referencia al original.
factory._create_model = getattr(factory, "create_model", None)

# Reemplazar en ambos lugares.
factory.create_model = patched_create_model
hook.create_model = patched_create_model

print(
    "Parche aplicado: CLAP_Module ahora reconoce 'music_audioset' como alias de 'HTSAT-base'"
)


def patched_load_state_dict(checkpoint_path, map_location="cpu"):
    """
    Modifica cómo se cargan los pesos (state_dict) del checkpoint.
    
    Patches factory.load_state_dict:
    1. Resuelve el error de seguridad (numpy global).
    2. Asegura que NO se salten los parámetros ('skip_params=False' implícito)
       para que se carguen los pesos de audio (HTSAT) y texto (RoBERTa).
    """

    # Define los globals requeridos por el checkpoint.
    safe_globals = ["numpy.core.multiarray.scalar"]

    print(
        f"✅ Aplicando parche de seguridad para cargar el checkpoint: {checkpoint_path}"
    )

    # 1. Usar el context manager para permitir los globals.
    # 2. Usar weights_only=False, como sugiere el error de PyTorch.
    with serialization.safe_globals(safe_globals):
        # Cargar el archivo de checkpoint completo.
        checkpoint = torch.load(
            checkpoint_path, map_location=map_location, weights_only=False
        )

    # Extraer el 'state_dict'. La mayoría de los checkpoints de PyTorch guardan los pesos aquí.
    if isinstance(checkpoint, dict) and "state_dict" in checkpoint:
        state_dict = checkpoint["state_dict"]
    else:
        # Si el checkpoint es solo el state_dict.
        state_dict = checkpoint

    return state_dict


factory.load_state_dict = patched_load_state_dict
print(
    "Patch aplicado: factory.load_state_dict modificado para carga segura y completa de pesos."
)

if not hasattr(np.random, "integers"):
    """ 
    Crea un alias integers → randint.
    """
    print("Aplicando parche de compatibilidad: np.random.integers -> np.random.randint")
    # Crear un alias para que las llamadas internas a 'integers' usen 'randint'.
    np.random.integers = np.random.randint

Parche aplicado: CLAP_Module ahora reconoce 'music_audioset' como alias de 'HTSAT-base'
Patch aplicado: factory.load_state_dict modificado para carga segura y completa de pesos.
Aplicando parche de compatibilidad: np.random.integers -> np.random.randint


In [6]:
def compute_clap_scores(
    results: list[MusicGenData], device=None
) -> list[MusicGenCLAPResult]:
    """
    Calcula el CLAP Score (similaridad texto-audio) usando embeddings del modelo CLAP.
    """
    device = device or ("cuda" if torch.cuda.is_available() else "cpu")
    print(f"\nUsando dispositivo: {device}\n")

    # 1. Cargar modelo CLAP
    clap_model = CLAP_Module(
        enable_fusion=True,
        amodel="HTSAT-base",  # Modelo de 1024 dims, compatible con el checkpoint con los pesos.
    )  # Activa la modalidad combinada audio-texto del modelo.

    state_dict = factory.load_state_dict(laion_clap_path, map_location=device)
    clap_model.model.load_state_dict(state_dict, strict=False)

    # clap_model.load_ckpt(laion_clap_path)  # Descarga y carga los pesos preentrenados.
    clap_model.eval()  # Modo evaluación (desactiva dropout, gradientes, etc.).
    clap_model.to(device)

    print("Modelo CLAP cargado correctamente.\nCalculando CLAP Scores...\n")

    scored: list[MusicGenCLAPResult] = []

    # 2. Iterar sobre los resultados
    for r in tqdm(results, desc="Procesando audios", ncols=80):
        try:
            audio, sr = torchaudio.load(r.audio_path)
            if sr != 48000:
                audio = torchaudio.functional.resample(audio, sr, 48000)
            audio = audio.to(device)

            with torch.no_grad():
                audio_emb = clap_model.get_audio_embedding_from_data(
                    audio, use_tensor=True
                )
                text_emb = clap_model.get_text_embedding(
                    [r.description], use_tensor=True
                )

                audio_emb = torch.nn.functional.normalize(audio_emb, dim=-1)
                text_emb = torch.nn.functional.normalize(text_emb, dim=-1)

                score = torch.nn.functional.cosine_similarity(
                    audio_emb, text_emb
                ).item()

            clap_score = round(float(score), 6)
            scored.append(
                MusicGenCLAPResult(
                    id=r.id,
                    taste=r.taste,
                    description=r.description,
                    instrument=r.instrument,
                    audio_path=r.audio_path,
                    clap_score=clap_score,
                )
            )

        except Exception as e:
            print(f"Error procesando {r.id}: {e}")

    return scored

In [15]:
# 1. Construir lista de audios generados existentes.
print(f"\nBuscando audios en: {tracks_base_data_path}")
audio_files = [f for f in os.listdir(tracks_base_data_path) if f.endswith(".wav")]

if not audio_files:
    raise FileNotFoundError(f"No se encontraron audios en {tracks_base_data_path}")

print(f"Se encontraron {len(audio_files)} archivos de audio.")

# Inferir metadata a partir del nombre de archivo.
results: list[MusicGenData] = []
for fname in audio_files:
    audio_path = os.path.join(tracks_base_data_path, fname)
    file_id = os.path.splitext(fname)[0]
    taste = file_id.split("_")[0] if "_" in file_id else "unknown"
    description = f"{taste} music, ambient for fine restaurant"

    results.append(
        MusicGenData(
            id=file_id,
            taste=taste,
            instrument="N/A",
            description=description,
            audio_path=audio_path,
        )
    )

print(f"Preparados {len(results)} registros para evaluación CLAP.\n")

# 2. Calcular CLAP Scores.
scored_results = compute_clap_scores(results, device=DEVICE)

# 3. Guardar resultados.
df = pd.DataFrame(scored_results)
df.to_csv(data_clap_path, index=False)
print(f"\nPipeline completo: resultados guardados en {data_clap_path}")


Buscando audios en: /home/juana/audio_reprompt/data/tracks/generated_base_music
Se encontraron 100 archivos de audio.
Preparados 100 registros para evaluación CLAP.


Usando dispositivo: cpu



Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


✅ Aplicando parche de seguridad para cargar el checkpoint: /home/juana/audio_reprompt/models/checkpoints/music_audioset_epoch_15_esc_90.14.pt
Modelo CLAP cargado correctamente.
Calculando CLAP Scores...



Procesando audios:   0%|                                | 0/100 [00:00<?, ?it/s]/home/juana/.venv311/lib64/python3.11/site-packages/torchaudio/_backend/utils.py:213: UserWarning: In 2.9, this function's implementation will be changed to use torchaudio.load_with_torchcodec` under the hood. Some parameters like ``normalize``, ``format``, ``buffer_size``, and ``backend`` will be ignored. We recommend that you port your code to rely directly on TorchCodec's decoder instead: https://docs.pytorch.org/torchcodec/stable/generated/torchcodec.decoders.AudioDecoder.html#torchcodec.decoders.AudioDecoder.
  warnings.warn(
/home/juana/.venv311/lib64/python3.11/site-packages/torchaudio/transforms/_transforms.py:581: UserWarning: Argument 'onesided' has been deprecated and has no influence on the behavior of this module.
  warnings.warn(
Procesando audios: 100%|██████████████████████| 100/100 [02:34<00:00,  1.54s/it]



Pipeline completo: resultados guardados en /home/juana/audio_reprompt/data/scores/results_with_clap_weights.csv


In [16]:
df

,id,instrument,taste,description,audio_path,clap_score
0,bitter_24,N/A,bitter,"bitter music, ambient for fine restaurant",/home/juana/audio_reprompt/data/tracks/generat...,0.066284
1,bitter_25,N/A,bitter,"bitter music, ambient for fine restaurant",/home/juana/audio_reprompt/data/tracks/generat...,0.059639
2,bitter_13,N/A,bitter,"bitter music, ambient for fine restaurant",/home/juana/audio_reprompt/data/tracks/generat...,0.051672
3,bitter_16,N/A,bitter,"bitter music, ambient for fine restaurant",/home/juana/audio_reprompt/data/tracks/generat...,0.057970
4,sour_25,N/A,sour,"sour music, ambient for fine restaurant",/home/juana/audio_reprompt/data/tracks/generat...,0.045256
...,...,...,...,...,...,...
95,salty_22,N/A,salty,"salty music, ambient for fine restaurant",/home/juana/audio_reprompt/data/tracks/generat...,0.060276
96,bitter_19,N/A,bitter,"bitter music, ambient for fine restaurant",/home/juana/audio_reprompt/data/tracks/generat...,0.071334
97,sweet_02,N/A,sweet,"sweet music, ambient for fine restaurant",/home/juana/audio_reprompt/data/tracks/generat...,0.070615
98,bitter_14,N/A,bitter,"bitter music, ambient for fine restaurant",/home/juana/audio_reprompt/data/tracks/generat...,0.030150


Comparación con los clap score con los pesos por defecto:

In [21]:
data_base_path = PROJECT_ROOT / config.data.data_clap_path / "results_with_clap_base.csv"

In [22]:
df_base = pd.read_csv(data_base_path)
df_base

,id,instrument,taste,description,audio_path,clap_score
0,sweet_01,NaN,sweet,"sweet music, ambient for fine restaurant",data/tracks/generated_base_music/sweet_01.wav,0.128623
1,sweet_02,NaN,sweet,"sweet music, ambient for fine restaurant",data/tracks/generated_base_music/sweet_02.wav,0.275660
2,sweet_03,NaN,sweet,"sweet music, ambient for fine restaurant",data/tracks/generated_base_music/sweet_03.wav,0.195981
3,sweet_04,NaN,sweet,"sweet music, ambient for fine restaurant",data/tracks/generated_base_music/sweet_04.wav,0.170296
4,sweet_05,NaN,sweet,"sweet music, ambient for fine restaurant",data/tracks/generated_base_music/sweet_05.wav,0.186923
...,...,...,...,...,...,...
95,salty_21,NaN,salty,"salty music, ambient for fine restaurant",data/tracks/generated_base_music/salty_21.wav,0.084582
96,salty_22,NaN,salty,"salty music, ambient for fine restaurant",data/tracks/generated_base_music/salty_22.wav,-0.039007
97,salty_23,NaN,salty,"salty music, ambient for fine restaurant",data/tracks/generated_base_music/salty_23.wav,-0.020779
98,salty_24,NaN,salty,"salty music, ambient for fine restaurant",data/tracks/generated_base_music/salty_24.wav,0.015667


### Conclusiones:

| Modelo                                   | Rango de valores             | Promedio general | Observaciones                                                           |
| ---------------------------------------- | ---------------------------- | ---------------- | ----------------------------------------------------------------------- |
| **Por defecto**                          | de **≈ -0.18** a **≈ +0.32** | **≈ 0.06**       | Gran dispersión; algunos negativos; sensibilidad desigual por sabor.    |
| **music_audioset_epoch_15_esc_90.14.pt** | de **≈ 0.027** a **≈ 0.093** | **≈ 0.061**      | Valores más homogéneos y positivos; rango estrecho; menor variabilidad. |

El modelo especializado produce scores más consistentes y todos positivos, aunque de magnitud más baja (≈0.03–0.09). Esto sugiere una calibración diferente del espacio de embeddings: menos extremos, pero más estables.

El modelo especializado mejora la “alineación semántica global” entre texto y audio, lo que sugiere una mayor robustez perceptiva para tareas de clasificación o evaluación contextual (e.g., emparejar música con conceptos).

Sin embargo, el modelo base es más sensible a diferencias de estilo, lo cual podría ser útil si se busca diferenciación emocional más marcada entre sabores.